# TRAVELING SALESMAN PROBLEM

The traveling salesman problem (also called the traveling salesperson problem or TSP) asks the following question: "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?"

In this project, I am going to solve different TSPs with different approaches. I have three different problem sets with 51 cities, 101 cities, and 130 cities.

I am going to use following approaches to solve the TSPs:
* Nearest Neighbor
* 2-OPT
* Nearest Neighbor + 2-OPT (Hybrid)
* Particle Swarm Optimization
* COTS Solution (Google OR Tools)

In this report, I am going to provide solutions, results, and their comparisons according to their accuracy and complexity. Also, I am going to include my comments & improvements for each algorithm. 

I am going to start with modeling and reading the data. For this purpose, I am going to read the data and create a City object with city number, X, and Y values in the each row. 

Let's start with City class then: 

In [ ]:
class City:
    """Class for City object. Attributes:
    -City ID
    -X coordinate
    -Y coordinate
    -Z value for Particle Swarm Optimization
    """
    def __init__(self, city_id, x, y):
        self.__city_id = city_id
        self.__x = x
        self.__y = y
        self.z = city_id  # particle swarm optimization parameter

    def __str__(self):
        return self.__city_id

    def __repr__(self):
        return self.__city_id

    def __eq__(self, other):
        return self.__city_id == other.__city_id

    def __lt__(self, other):
        return self.z < other.z

    def __sub__(self, other):
        self.z -= other.z
        return self

    def __add__(self, other):
        self.z += other.z
        return self

    def scale(self, scalar):
        """Scales the City's z parameter with the given scalar value."""
        self.z = self.z * scalar
        return self

    def get_city_id(self):
        return self.__city_id

    def get_x(self):
        return self.__x

    def get_y(self):
        return self.__y

    def distance(self, other):
        """Calculates the Euclidean distance between two cities."""
        return round(((self.__x - other.__x) ** 2 + (self.__y - other.__y) ** 2) ** (1/2))

After that, I am going to the read the data, create the City objects, and store them in a list. For this purpose, I am going to use a helper function, txt_reader.

In [ ]:
def txt_reader(file_name):
    """Reads the given input files and store the cities."""
    city_list = []
    txt_file = open(file_name, "r")
    txt_file.readline()  # skip the header line
    for line in txt_file:
        temp_list = line.rstrip().split(" ")
        temp_city = City(temp_list[0], int(float(temp_list[1])), int(float(temp_list[2])))
        city_list.append(temp_city)
    return city_list

Now, I can read the data and model it. 

Before focusing on the solutions, I am going to implement other helper functions, calc_path_distance & visualize_route. I am going to use these functions to calculate the distance of the route and visualize the route. 

In [ ]:
def calc_path_distance(city_list):
    """Calculates the distance of the TSP route."""
    path_distance = 0
    for i in range(len(city_list) - 1):
        path_distance += city_list[i].distance(city_list[i + 1])
    path_distance += city_list[-1].distance(city_list[0])  # adds the arc between starting city & ending city.
    return path_distance

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


def visualize_route(title, route):
    """Visualizes the route for the TSP."""
    fig = plt.figure()
    fig.suptitle(title)
    x = []
    y = []
    for city in route:
        x.append(city.get_x())
        y.append(city.get_y())
    x.append(route[0].get_x())
    y.append(route[0].get_y())
    graph, = plt.plot(x, y, 'ko')
    graph, = plt.plot(x, y, 'darkorange')
    
    # Since I can not use animation on Jupyter Notebook, this part is commented.
    #def animate(i):
    #    graph.set_data(x[:i + 1], y[:i + 1])
    #    return graph

    #ani = FuncAnimation(fig, animate, frames=len(route) + 10, interval=500)
    plt.show()
    plt.show(block=True)

Here, all the helper functions are implemented. I am going to start with the first solution, which is Nearest Neighbor.

## Nearest Neighbor

The nearest neighbour algorithm was one of the first algorithms used to solve the travelling salesman problem approximately. In that problem, the salesman starts at a random city and repeatedly visits the nearest city until all have been visited. The algorithm quickly yields a short tour, but usually not the optimal one.

The algorithm can be summed up with the following steps:

1. Initialize all vertices as unvisited.
2. Select an arbitrary vertex, set it as the current vertex u. Mark u as visited.
3. Find out the shortest edge connecting the current vertex u and an unvisited vertex v.
4. Set v as the current vertex u. Mark v as visited.
5. If all the vertices in the domain are visited, then terminate. Else, go to step 3.

I am going to implement this algorithm in three parts. 

Firstly, I am going to implement a function which takes a city and a city list as input; and returns the nearest city to the given city in the given city list.

In [ ]:
from sys import maxsize


def nearest_neighbor(city, city_list):
    """Finds the nearest neighbor of a given city."""
    min_distance = maxsize
    nearest_city = None
    for c in city_list:
        temp_distance = city.distance(c)
        if temp_distance < min_distance:
            min_distance = temp_distance
            nearest_city = c
    return nearest_city, min_distance

Secondly, I am going to implement a function, which takes a starting city and city list as input; and returns a route (with the nearest neighbor principle) and its distance.

In [ ]:
def nearest_neighbor_algorithm(next_city, city_list):
    """Creates a route by starting with the given city & following the nearest neighbors."""
    copy_city_list = city_list[:]
    path = [next_city]
    copy_city_list.remove(next_city)
    while copy_city_list:
        next_step = nearest_neighbor(next_city, copy_city_list)
        next_city = next_step[0]
        path.append(next_city)
        copy_city_list.remove(next_city)
    path_distance = calc_path_distance(path)
    path.append(path[0])  # add starting city to route, to create a cycle.
    return path, path_distance

Thirdly and lastly, I am going to implement a function to try all starting city alternatives for the previous function. Simply, it makes an iteration for each starting city. This function takes a city list, and returns the best route and its distance among all alternatives.

In [ ]:
def nna_iteration(city_list):
    """Finds the best route by changing the starting city."""
    best_path = []
    best_distance = maxsize
    for city in city_list:
        result = nearest_neighbor_algorithm(city, city_list)
        if best_distance > result[1]:
            best_path = result[0]
            best_distance = result[1]
    return best_path, best_distance

Now, I can try the Nearest Neighbor Algorithm with 51 cities.

In [ ]:
import time


city_list = txt_reader("data/51_Cities.txt")

t0 = time.time()
nna_result = nna_iteration(city_list)
t1 = time.time()
print("Nearest Neighbor Path: " + str(nna_result[0]) + "\nNearest Neighbor Solution: " + str(nna_result[1]) + "\nNearest Neighbor Elapsed Time: " + str(round((t1 - t0), 3)) + " seconds")
# visualize_route("Nearest Neighbor --- Solution: " + str(nna_result[1]), nna_result[0])

*Since, I cannot use animation on Jupyter Notebook, I am going to add a .gif for the animated result (51 Cities) for this and other solutions. 
You can run the shared source code on any IDE to see the live animated result or uncomment the visualize_route function to see the final result.*

### Nearest Neighbor Result

Result of the Nearest Neighbor can be seen below:

<img align="left" width="640" height="480" src="media/nna.gif">

**Nearest Neighbor Path:** [8, 26, 31, 28, 3, 20, 35, 36, 29, 21, 50, 9, 49, 5, 38, 11, 32, 1, 22, 2, 16, 34, 30, 10, 39, 33, 45, 15, 44, 37, 17, 4, 18, 47, 12, 46, 51, 27, 6, 48, 23, 7, 43, 24, 14, 25, 13, 41, 19, 42, 40, 8]

**Nearest Neighbor Solution:** 482

**Nearest Neighbor Elapsed Time:** 0.101 seconds

So, there is 13.1% error compared the optimal solution; but for 51 cities, running time of this algorithm is 0.101 seconds.

As it seen on the .gif, the algorithm creates the route in a greedy way, and there are multiple knots. The algorithm can be improved with following approaches:

1. Untangling these knots. (Later on I am going to fuse another algorithm (2-OPT) with Nearest Neighbor to accomplish this.)
2. Being less greedy with considering the consequences. 

## 2-OPT

In optimization, 2-OPT is a simple local search algorithm for solving the traveling salesman problem. The main idea behind it is to take a route that crosses over itself and reorder it so that it does not. 

The algorithm can be summed up with the following steps:

Let's say i & j are indexes.

1. Take route[0] to route[i-1] and add them in order to new_route
2. Take route[i] to route[j] and add them in reverse order to new_route
3. Take route[j+1] to end and add them in order to new_route

A complete 2-OPT local search will compare every possible valid combination of the swapping mechanism. This technique can be applied to the travelling salesman problem as well as many related problems.

I am going to implement this algorithm in two parts.

Firstly, I am going to implement a function which takes a route, i & j; and returns the new_route.

In [ ]:
def two_opt_swap(route, i, j):
    """Finds an alternative route of the given route with 2-OPT."""
    new_list1 = route[0:i]
    new_list2 = route[i:j]
    new_list2.reverse()
    new_list3 = route[j:]
    return new_list1 + new_list2 + new_list3

Secondly and lastly, I am going to implement a function to compare every possible valid combination of the swapping mechanism. This function takes a route; and returns the best route and its distance among all alternatives.

In [ ]:
from itertools import combinations


def two_opt(route):
    """Tries all alternative 2-OPT routes of the given route."""
    copy_route = route[:]
    index_list = []
    best = calc_path_distance(copy_route)
    for i in range(len(copy_route)):
        index_list.append(i)
    comb_index = list(combinations(index_list, 2))  # all alternative 2-OPTs
    i = 0
    while i < len(comb_index):
        temp_list = (two_opt_swap(copy_route, comb_index[i][0], comb_index[i][1]))
        if calc_path_distance(temp_list) < best:
            copy_route = temp_list
            best = calc_path_distance(temp_list)
            i = 0
        i = i + 1
    copy_route.append(copy_route[0])  # add starting city to route, to create a cycle.
    return copy_route, best

Now, I can try the 2-OPT Algorithm with 51 cities.

In [ ]:
t2 = time.time()
two_opt_result = two_opt(city_list)
t3 = time.time()
print("2-OPT Path: " + str(two_opt_result[0]) + "\n2-OPT Solution: " + str(two_opt_result[1]) + "\n2-OPT Elapsed Time: " + str(round((t3 - t2), 3)) + " seconds")
#visualize_route("2-OPT --- Solution: " + str(two_opt_result[1]), two_opt_result[0])


### 2-OPT Result

Result of the 2-OPT can be seen below: 

<img align="left" width="640" height="480" src="media/2-opt.gif">


**2-OPT Path:** [46, 32, 1, 22, 28, 3, 36, 35, 20, 2, 11, 38, 5, 49, 9, 50, 16, 29, 21, 34, 30, 10, 39, 33, 45, 15, 44, 37, 17, 42, 40, 19, 41, 13, 4, 12, 47, 18, 25, 14, 6, 24, 43, 7, 23, 48, 8, 26, 31, 27, 51, 46]

**2-OPT Solution:** 461

**2-OPT Elapsed Time:** 2.416 seconds

So, there is 8.2% error compared the optimal solution and running time of this algorithm is 2.416 seconds for 51 cities. Note that, initial route affects the running time, and same route given to the both algorithm. 

It can be said that, it is better than the Nearest Neighbor, but it is much slower.

As it seen on the .gif, there are no knots in this algorithm; but it creates the route without considering any nearest path. So the algorithm can be improved with the following approaches:

1. Feeding it with the Nearest Neighbor output, so it can also be greedy.
2. Increasing the number of deleted edges, such as **3-OPT**, but it will be slower. 

## Nearest Neighbor + 2-OPT (Hybrid)

Since I have these two algorithms, and they cover up each others weak parts; now I am going to use Nearest Neighbor's output as an input to 2-OPT.  

In [ ]:
t4 = time.time()
nna_result2 = nna_iteration(city_list)
hybrid_result = two_opt(nna_result2[0])
hybrid_result[0].pop()
t5 = time.time()
print("Nearest Neighbor + 2-OPT (Hybrid) Path: " + str(hybrid_result[0]) + "\nHybrid Solution: " + str(hybrid_result[1]) + "\nHybrid Elapsed Time: " + str(round((t5 - t4), 3)) + " seconds")
#visualize_route("Nearest Neighbor + 2-OPT (Hybrid) --- Solution: " + str(hybrid_result[1]), hybrid_result[0])

### Nearest Neighbor + 2-OPT (Hybrid) Result

Result of the Hybrid can be seen below:

<img align="left" width="640" height="480" src="media/hybrid.gif">

**Hybrid Path:** [8, 26, 31, 28, 3, 36, 35, 20, 29, 2, 16, 50, 21, 34, 30, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 49, 9, 38, 11, 32, 1, 22, 8]

**Hybrid Solution:** 428

**Hybrid Elapsed Time:** 0.817 seconds

So, there is 0.4% error compared the optimal solution and running time of this algorithm is 0.817 seconds for 51 cities.

Since 2-OPT is feeded with an already good solution, it ran in a much shorter time compared the previous execution.

## Particle Swarm Optimization

In computational science, particle swarm optimization (PSO) is a computational method that optimizes a problem by iteratively trying to improve a candidate solution with regard to a given measure of quality. 

It solves a problem by having a population of candidate solutions, here dubbed particles, and moving these particles around in the search-space according to simple mathematical formula over the particle's position and velocity.

Each particle's movement is influenced by its local best known position, but is also guided toward the best known positions in the search-space, which are updated as better positions are found by other particles. This is expected to move the swarm toward the best solutions.

Equation for the Particle Swarm Optimization can be seen below:

<img align="left" src="media/pso_eq.png">

And a visual representation (a particle swarm searching for the global minimum of a function) of the algorithm can be seen below:

<img align="left" src="media/pso_vis.gif">

To understand better, an analogy between the algorithm and bird swarms can be used. Bird swarms look for the food randomly, and when one of the birds find food, other birds will look for the food around it.

To implement this algorithm, I have added a Z value to the City object, which represents the index of a city in a route. Each particle will have their own route, and they are going to minimize the distance by sharing their best personal routes in each iteration. 

I have implemented components of the equation and decided on the parameters with trying different parameters of different PSO variations.

In [ ]:
import random


t0 = time.time()

# Constants
c1 = 1.494  # self confidence
c2 = 1.494  # swarm confidence
w = 0.729  # inertia weight

# Iteration number and swarm size
maxIter = 1000
swarmSize = 500

# lists for particles, their positions, and their velocities.
p = []  # particles
x = []  # positions of the particles
v = []  # velocities of the particles

best = [maxsize for i in range(swarmSize)]  # best values for each particle, maxsize at inital
g = 0  # index of the global best

org_city_list = txt_reader('data/51_Cities.txt')
num_city = len(org_city_list)

for i in range(swarmSize):
    route1 = []
    route2 = []
    route3 = []
    for c in range(num_city):
        org_city = org_city_list[c]
        city1 = City(org_city.get_city_id(), org_city.get_x(), org_city.get_y())
        city2 = City(org_city.get_city_id(), org_city.get_x(), org_city.get_y())
        city3 = City(org_city.get_city_id(), org_city.get_x(), org_city.get_y())
        rand = random.random() * 50

        # added a random value to create variation between initial routes of particles
        city1.z = rand + i
        city2.z = rand + i
        city3.z = 0  # velocity is zero at the beginning.

        route1.append(city1)
        route2.append(city2)
        route3.append(city3)

    p.append(route1)
    x.append(route2)
    v.append(route3)

for n in range(maxIter):

    # algorithm can be improved by using a dynamic inertia weight
    # if n < maxIter/2:
    #     w = (0.85 - 0.55) * (maxIter/2 - n) / (maxIter/2) + 0.55
    # else:
    #     w = (0.85 - 0.55) * (maxIter - n) / (maxIter/2) + 0.55
    # w = min(max(w + 0.0004,0.69),0.8)

    for i in range(swarmSize):

        res = calc_path_distance(sorted(x[i]))
        if res < min(best):  # finds the global best
            for c in range(num_city):
                p[g][c].z = x[i][c].z
            g = i

            print('New cost: ', str(res))
        if res < best[i]:  # finds the personal best
            best[i] = res
            for c in range(num_city):
                p[i][c].z = x[i][c].z
        for c in range(num_city):  # finds the next velocity and position of the particles
            rand1 = random.random()
            rand2 = random.random()
            v[i][c].z = w * v[i][c].z + c1 * rand1 * (p[i][c].z - x[i][c].z) + c2 * rand2 * (p[g][c].z - x[i][c].z)  #next velocities
            x[i][c].z = x[i][c].z + v[i][c].z  # next positions

p[g].sort()  # sort the particles by their z value.
p[g].append(p[g][0])  # add starting city to route, to create a cycle.

t1 = time.time()

print("Particle Swarm Optimization Path: " + str(p[g]) + "\nParticle Swarm Optimization Solution: " + str(calc_path_distance(p[g])) + "\nParticle Swarm Optimization Elapsed Time: " + str(round((t1 - t0), 3)) + " seconds")
#visualize_route("Particle Swarm Optimization --- Solution: " + str(calc_path_distance(p[g])), p[g])

### Particle Swarm Optimization Result

Result of the Particle Swarm Optimization can be seen below:

<img align="left" width="640" height="480" src="media/pso_best.gif">

**Particle Swarm Optimization Path:** [31, 26, 7, 23, 48, 1, 32, 11, 38, 51, 12, 4, 18, 25, 13, 41, 19, 40, 42, 44, 17, 37, 15, 45, 33, 39, 10, 30, 9, 49, 5, 47, 46, 27, 6, 14, 24, 43, 8, 22, 2, 16, 50, 34, 21, 29, 20, 35, 36, 3, 28, 31]

**Particle Swarm Optimization Solution:** 489

**Particle Swarm Optimization Elapsed Time:** 138.369 seconds

Max. iteration is 1000, swarm size is 500 for this execution. 

*Note that; path, solution, and running time can change in other execution since particles are starting and moving randomly.*

For this execution, there is 14.7% error compared the optimal solution and running time of this algorithm is 138.369 seconds for 51 cities. It can be said that, this algorithm did not performed as good as the previous ones.

The algorithm can be improved with the following approaches:

1. A dynamic (according to iteration) inertia weight can be used. (Please check the commented inertia weight part in the code.)
2. Max. iteration number and swarm size can be fine tuned.
3. Algorithm can be fed with another algorithms output, so particles can have their starting routes with good ones, instead of random ones.

## COTS Solution (Google OR Tools)

These are the Commercial-of-the-shelf solutions provided by Google OR Tools. I am not going to include codes on here, I only coded a function to fit our data to the solution, you can see it on the shared source code, google_or.py.

In this part, I have explored the results with different parameters; such as First Solution Strategy parameters and Local Search options parameters, etc.

Almost all the parameters in the shared link are explored: https://developers.google.com/optimization/routing/routing_options

I am going to include some of the results to the report.

### COTS Solution (Google OR Tools) Result

#### Guided Local Search Result (Meta-Heuristic)

<img align="left" width="640" height="480" src="media/gls.gif">

**Google OR (GUIDED LOCAL SEARCH) Path:** [1, 32, 11, 38, 5, 37, 17, 4, 18, 47, 12, 46, 51, 27, 6, 48, 23, 7, 43, 24, 14, 25, 13, 41, 40, 19, 42, 44, 15, 45, 33, 39, 10, 49, 9, 30, 34, 50, 16, 21, 29, 2, 20, 35, 36, 3, 28, 31, 26, 8, 22, 1]

**Google OR (GUIDED LOCAL SEARCH) Solution:** 426

**Time Limit:** 30 seconds

It gives the best solution, which is 426 and there is no error for 51 cities

#### Savings Result (Heuristic)

<img align="left" width="640" height="480" src="media/savings.gif">

**Google OR (SAVINGS) Path:** [1, 27, 6, 48, 23, 24, 43, 7, 26, 8, 31, 28, 3, 36, 35, 20, 29, 21, 16, 38, 5, 49, 9, 50, 34, 30, 10, 39, 33, 45, 15, 44, 42, 19, 40, 41, 13, 25, 14, 18, 4, 17, 37, 12, 47, 51, 46, 32, 11, 2, 22, 1] 

**Google OR (SAVINGS) Solution:** 436

For 51 cities, there is 2.3% error compared to optimal solution.

## Conclusion



In this project, I have solved different TSPs with different approaches. I had three different problem sets with 51 cities, 101 cities, and 130 cities. For each different problem sets, I have used following approaches and tried to deeply understand each one of them. 

* Nearest Neighbor
* 2-OPT
* Nearest Neighbor + 2-OPT (Hybrid)
* Particle Swarm Optimization
* COTS Solution (Google OR Tools)

For each approaches, I have reported the results and added my comments to how to improve them. 

For the final part of this report, I am going to include a table which includes the comparisons of these approaches. 

*Note that, bold ones are the algorithms I have implemented, others are COTS solutions provided by Google. Algorithms are tested with 51 cities.*

You can see the final comparison below: 

| Algorithm | Solution | Error |
| --- | --- | --- |
| Guided Local Search | 426 | 0 |
| **Nearest Neighbor + 2-OPT (Hybrid)** | 428 | .46 |
| Path Most Constrained Arc | 430 | .93 |
| Tabu Search | 430 | .93 |
| Global Cheapest Arc | 432 | 1.40 |
| First Unbound Min. Value | 433 | 1.64 |
| Local Cheapest Arc | 435 | 2.11 |
| Savings | 436 | 2.34 |
| Simulated Annealing | 438 | 2.81 |
| Generic Tabu Search | 438 | 2.81 |
| Path Cheapest Arc | 439 | 3.05 |
| Greedy Descent | 439 | 3.05 |
| Local Cheapest Insertion | 442 | 3.75 |
| Christofides | 449 | 5.39 |
| Parallel Cheapest Insertion | 450 | 5.63 |
| **2-OPT** | 461 | 8.21 |
| **Nearest Neighbor** | 482 | 13.14 |
| **Particle Swarm Optimization** | 489 | 14.78 |